# Implementing linear regression with NumPy

In this notebook we implement linear regression with NumPy, to train models on the on the Portland housing dataset.

The following exercises need to be completed for week 5.



**Beware** The changes that you make to this on-line notebook will not be saved. As soon as you close or lose this tab, your answers will disappear.

**Download this notebook to you local disk regularly**  

**Bring a copy** of the finished exercises to the next class. Your notebook will be graded by the teacher.

## The portland dataset

### Load Portlant housing data

Run the cells below to load the Portland dataset.  
*Hint: press shift+enter to evaluate cells.  
The menu Help -> Keyboard shortcuts lists more shortcuts*

In [ ]:
%matplotlib inline
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
import tensorflow as tf
sns.set_context("poster")
plt.style.use('fivethirtyeight')
matplotlib.style.use('fivethirtyeight') 
plt.rcParams['figure.figsize'] = (10, 6)
scatter_size = 60

portland = pd.read_csv('portlant_housing.txt', header=None, names=['area', 'bedrooms', 'price'])
portland.price = (portland.price / 1000)

### Array and DataFrame
The variable `portland` is a so called Pandas `DataFrame`. Pandas is a module for working with structured data. Don't worry if you are not familiar with about Pandas. Pandas provides extra functionality on top of NumPy. Behind every pandas `DataFrame` lies a NumPy `ndarray`. `Dataframe` has some methods in common with `ndarray`. Furthermore, NumPy functions accept `DataFrame` type arguments.


1. Get the type of the variable portland
1. Get a listing of the first 10 houses by making use of `portland.head()`.  
 *Hint: use the help function to learn more about df.head()*
5. Extract the underlying NumPy array of `portland` by calling  `portland.values`, and slice the array to show the first 5 rows only. Use brackets `[]` for the slicing.  
*Hint: remember the values attribute for later exercises where you need a NumPy `ndarray` instead of a Pandas DataFrame.*
5. Subsetting works slightly different with Pandas. Use 'portland.iloc' to get the last three rows of the dataframe.  
*Hint: use the help function on 'portland.iloc'*

### Inspecting the Portland dataset

Subsetting works similar on `DataFrames`

1. How many data points are in the DataFrame `portland`? Use `portland.shape`.
2. What are the mean, minimum and maximum of the number of bedrooms and of the price? Use `portland.mean()`, `portland.max()` and `portland.min()`.
3. Make a scatter plot of the dataset by using the code block below

In [ ]:
plt.figure(figsize = (15,4),dpi=100)
ax0 = plt.subplot(121)
portland.plot('area', 'price', kind='scatter', ax=ax0, s=scatter_size);
plt.xlabel("Size of house (x_1)")
plt.ylabel("Price (Y)")
ax1 = plt.subplot(122)
portland.plot('bedrooms', 'price', kind='scatter', ax=ax1, s=scatter_size);
plt.xlabel("Number of Bedrooms (X_2)")
plt.ylabel("Price (Y)");

## Loss functions

### Square loss with pure Python
1. Use pure python to write mean square loss function. Do not use NumPy or any other library for the calculation.

In [ ]:
def square_loss_python(h, y):
    """
    Returns the mean square loss of h and y
    h: list of floats
    y: list of floats
    return: float
    """
    pass

# Check answer
square_loss_python([3., 4., 5.], [2., 1, 1]) == 26/3

In [ ]:
# %load answers/square_loss_python.py

### Square loss with Numpy
2. Use `numpy.dot` to write the mean square loss function.

In [ ]:
def square_loss(h, y):
    """
    Returns the mean square loss of h and y
    h: numpy vector
    y: numpy float
    return: float
    """
    pass

# Check answer
square_loss(np.array([3., 4., 5.]), np.array([2., 1, 1])) == 26/3

In [ ]:
# %load answers/square_loss.py

### A simple hypothesis

1. The simplest hypothesis we can come up with, is 
$$
h_0(x) = \theta_0 \,,
$$
a constant. What would be a reasonable value for $\theta$?
2. Estimate the mean square loss for this constant hypothesis, based on the plots above. Explain you reasoning.
2. Create a vector $h$ for the Portland dataset, based on your hypothesis.  
*Hint: make use of use of `np.ones', np.ones_like`.
3. Use the `square_loss()` to calculate the loss of $h_0$ on the Portland dataset
4. Consider the set of constant hypothesis,
$$
\mathcal H_\text{constant} = \left\{h_\theta(\mathbf x) = \theta_0\,, \theta_0 \in \mathbb R \right\} \,.
$$
Under these Hypothesis, the loss function becomes
$$
J(\theta_0) = \frac{1}{N} \sum_{i=1}^N (y_i - \theta_0 )^2 \,.
$$
We can find the analytical minimum of the cost function, by setting the derivative with respect to $\theta_0$ to zero. 
$$
\begin{align}
\frac{\partial}{\partial \theta_0}  J(\theta_0) & = \frac{2}{N} \sum_{i=1}^N( y_i - \theta_0)  \\
& = \frac{2}{N} \sum_{i=1}^N y_i - 2 \theta_0  \\
& = 0  \\
\end{align}
$$
Solve this equation for $\theta_0$.
Explain the solution in words
6. Use NumPy to compute the optimal value for $\theta_0$ for the Portland dataset



In [ ]:
# %load answers/constant_hypothesis.py

### Square loss function with TensorFlow

1. Complete the function 'square_loss_tensor' to calculate the loss with TensorFlow. The expected output is an abstract TensorFlow array. In other words: you do not have to evaluate the loss within this function. 
2. Use TensorFlow to evaluate the mean square loss for your constant hypothesis in the last exercise. Add comments in you code, and include the following parts
    - Python variables
    - Setup a computation Graph
    - Create place holders
    - Use your function `square_loss_tensor`
    - Create a session
    - Perform the calculation
    - Present the result  
You do not have to wrap your solution into a function.
3. Compare the loss of the TensorFlow implementation with the Python implementation. Are they identical?

In [ ]:
def square_loss_tensor(h, y):
    """
    Construct a node that computes the mean square loss as function of nodes h and y.

    Inputs:
    - h: TensorFlow tensor of dimension N with predicted outcomes for y
    - y: TensorFlow tensor of dimension N with actual outcomes for y

    Returns: TensorFlow scalar of type float 
        this is an abstract tensor of rank 0
    """
    return tf.constant(0) # Replace this

In [ ]:
# %load answers/square_loss_tensor.py

In [ ]:
# Check answer
h_tensor = tf.constant([3., 4., 5.], name="h")
y_tensor = tf.constant([2., 1, 1], name="y")
with tf.Session() as sess:
    print(sess.run(square_loss_tensor(h_tensor, y_tensor)))
    # should be 8.666....

## Linear regression with Python

### Design matrix

In linear regression, we work with the set of hypothesis 

$$ 
\mathbf h_\theta (X) = X \theta
$$

1. What is in the rows of $X$?
2. What is in the columns of $X$?
2. What is the bias term? And why is it missing in the Hypothesis?
2. Run the two cells below. Explain how it adds a bias term to the Hypothesis. What happens with the column price?

In [ ]:
portland.head()

In [ ]:
design_matrix = portland.copy()
design_matrix['bias'] = 1
design_matrix = design_matrix.drop('price', axis=1)
design_matrix.head()

### Implement the Hypothesis
Above, you have defined a constant hypothesis. Now it is time to take the independent variables into account. 


1.1. What are the independent variables / features in this problem?
1. What would be a reasonable guess for theta, when you take all independent variables into account?
1. Adapt the NumPy vector `theta_test` with your guess for $\mathbf \theta$.
1. Implement the linear hypothesis by completing the function `linear_regession_hypothesis`  
*Hint: you may want to check the Numpy documentation to find the function you need*
1. Apply the function `linear_regession_hypothesis` to the Portland dataset at `theta_test`
1. Does your regression implementation gives a reasonable outcomes?
1. Calculate the errors with `error = linear_hypothesis(X, theta_test) - y`. What is the mean of the errors? Do you see reason to adapt your hypothesis?

In [ ]:
theta_test = np.array([0, 0, 0])
X = design_matrix.values
y = portland.price.values

def linear_hypothesis(X_batch, theta):
    """
    Use the weights theta to predict outcomes for the data points X_batch.

    Inputs:
    - X_batch: A numpy array of shape (N, p) containing a minibatch of N
      data points; each point has dimension p.
    - theta: A numpy array of shape (p,) containing the weights of the hypothesis

    Returns: A tuple containing:
    - loss as a single float
    - gradient with respect to the parameter vector \theta
    
    Returns:
    - y_pred: Predicted outcomes for the data in X. y_pred is a 1-dimensional
      array of length N
    """
    
linear_hypothesis(X, theta_test)

In [ ]:
# %load answers/linear_hypothesis.py

### Update rules for linear regression

The update rules for gradient decent are

$$\mathbf \theta_j := \mathbf \theta_{j-1} - \alpha \left. \nabla_\theta J(\theta) \right\rvert_{\theta = \theta_{j-1}}\,,$$
where $J(\mathbf \theta)$ is the loss function. The exact from of the update rules depend on the model and the loss function that is used. The gradient of the mean square error for the linear hypothesis above equals

$$
\nabla_\theta J(\theta) = \frac{2}{N} \mathbf X^\top (\mathbf h_{\theta} - \mathbf y) \,.
$$



1. What is the meaning of the index $j$?
2. What are the dimensions of $\mathbf X^\top$, $\mathbf y$ and $\mathbf \theta$?
4. What is the interpretation of $\mathbf h_{\theta_{j-1}} - \mathbf y$?




### Gradients for linear regression

You are going to implement the gradients for linear regression using NumPy. The function `linear_regression_gradient` has been provided below.

2. Finish `linear_regression_gradient`
    1. For now you can ignore regularization parameter `reg`
    1. Make use of your function `linear_hypothesis` the calculate the predictions
    1. Make use of your function `square_loss(h, y)` to calculate the loss
    1. Make use of `X_batch.transpose()` and `np.matmul` to calculate the gradient. 
3. Compute the gradient for the Portland dataset with $\theta$ = `theta_test`, which you defined above

In [ ]:
def linear_regression_gradient(X_batch, y_batch, theta, reg=0):
    """
    Compute the loss function and its derivative. 

    Inputs:
    - X_batch: A numpy array of shape (N, p) containing a minibatch of N
      data points; each point has dimension p.
    - y_batch: A numpy array of shape (N,) containing labels for the minibatch.
    - theta: A numpy array of shape (p) of model parameters
    - reg: regularization strength

    Returns: A tuple containing:
    - loss as a single float
    - gradient with respect to the parameter vector \theta
    """
    loss = None
    gradient = None
    return (loss, gradient)

loss, gradient = linear_regression_gradient(X, y, np.array([0,0,0])
print(loss) # 131183.09621291485
print(gradient) # [-1.52841826e+06, -2.24073540e+03, -6.80825319e+02]

In [154]:
# %load answers/linear_regression_gradient.py

In [ ]:
linear_regression_gradient(X, y, theta_test)

### Scaling

1. Looking at the order of magnitude of the gradients for `theta_test`, what do you observe?
1. Why do we need to scale the design matrix?
1. Run the code below to scale the design matrix
1. Evaluate the gradient again, now on with `theta_test_scaled`, which is given below.
1. Do you still see a problem with the gradient?

In [ ]:
design_matrix_scaled = (design_matrix - design_matrix.mean())/(design_matrix.std())
design_matrix_scaled.bias = 1
design_matrix_scaled.head()

In [ ]:
theta_test_scaled = [110, -5, 340]
linear_regression_gradient(design_matrix_scaled.values, portland.price.values, theta_test_scaled)

### Implementing gradient descent
Below you can find code for the class `Regression`. 
1. What happens in the __init__() method?
1. How does `theta` gets initialized, eventually?
1. Complete the method `predict`
   1. Use `linear_hypothesis` that you have written before
   1. For `theta`, use `self.theta`
1. Complete the method `train` by updating the weights

In [ ]:
class LinearRegression(object):

    def __init__(self, reg=0):
        self.theta = None
        self.loss_history = []
        self.reg = reg

    def train(self, X, y, learning_rate=1e-3, num_iters=100,
        batch_size=None, verbose=False):
        """
        Train this linear model using stochastic gradient descent.

        Inputs:
        - X: A numpy array of shape (N, p) containing training data; there are N
          training samples each of dimension p.
        - y: A numpy array of shape (N,) containing training labels
        - learning_rate: (float) learning rate for optimization.
        - reg: (float) regularization strength.
        - num_iters: (integer) number of steps to take when optimizing
        - batch_size: (integer) number of training examples to use at each step.
        - verbose: (boolean) If true, print progress during optimization.

        Outputs:
        A list containing the value of the loss function at each training iteration.
        """
        num_train, p = X.shape
        if self.theta is None:
          # initialize weight vector theta with random values
          self.theta = 0.001 * np.random.randn(p)

        # Stochastic gradient descent to optimize theta
        loss_history = []
        for it in range(num_iters): # training iterations
            X_batch = None
            y_batch = None
            if batch_size:
                  batch = np.random.choice(len(X), batch_size)
                  X_batch = X[batch, :]
                  y_batch = y[batch]
            else:
                X_batch = X
                y_batch = y
            
              # Evaluate loss and gradient
            loss, grad = self.loss(X_batch, y_batch, self.reg)
            self.loss_history.append(loss)

            # perform parameter update
            #########################################################################
            # TODO:                                                                 #
            # Update the weights using the gradient and the learning rate.          #
            #########################################################################
            
            #########################################################################
            #                       END OF YOUR CODE                                #
            #########################################################################

            if verbose and (it + 1) % 20 == 0:
                print('iteration %d / %d: loss %f' % (it+1, num_iters, loss))
                #print('theta', self.theta)
                #print('grad', grad)
                #print('upate', -learning_rate * grad)

          
        return 

    def predict(self, X):
        """
        Use the trained weights of this linear model to predict labels for
        data points.

        Inputs:
        - X: A numpy array of shape (N, p) containing training data; there are N
          training samples each of dimension p.

        Returns:
        - y_pred: Predicted values for the data in X. y_pred is a 1-dimensional
          array of length N
        """
        y_pred = np.zeros(X.shape[0])
        # Make prediction
        #########################################################################
        # TODO:                                                                 #
        # Use the learned coefficients to make a prediction                     #
        #########################################################################
        
        #########################################################################
        #                       END OF YOUR CODE                                #
        #########################################################################
        
        return y_pred

    def loss(self, X_batch, y_batch, reg):
        """
        Compute the loss function and its derivative. 

        Inputs:
        - X_batch: A numpy array of shape (N, D) containing a minibatch of N
          data points; each point has dimension D.
        - y_batch: A numpy array of shape (N,) containing labels for the minibatch.
        - reg: (float) regularization strength.

        Returns: A tuple containing:
        - loss as a single float
        - gradient with respect to self.W; an array of the same shape as W
        """
        loss, grad = linear_regression_gradient(X_batch, y_batch, self.theta, reg)
        return (loss, grad)

    def plot_loss(self):
        if self.loss_history:
            plt.plot(self.loss_history)
            plt.title('Learning curve')
            plt.xlabel('Mini batches')
            plt.ylabel('Loss')

In [ ]:
# %load answers/linear_regression_class.py

### Tune the learning rate

After completing the class 'LinearRegression', you can train it on the Portland dataset.
1. Run the code below to train the model an view the learning curve
2. Adapt the learning rate and the number of iterations to bring down the loss. Keep record of your trials.
3. What are the coefficients that are learned?

In [ ]:
regression = LinearRegression()
regression.train(design_matrix_scaled.values, portland.price.values, batch_size=None, learning_rate=1e-2, num_iters=100, verbose=True)
regression.plot_loss()

In [ ]:
# %load answers/training_portland.py